In [ ]:
"""
Tree-of-Thought prompting
"""

In [1]:
# define variables
NUM_FILES = 20
FOLDER_NAME = '../../determining_files_rename/random_samples_functions'
GPT_SAVED_FILE_NAME = 'rename_function_gpt'

In [2]:
# read in files from folder random_cells 
random_cells = []

# read in cells
for i in range(NUM_FILES):
    file_name = f'{FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

# read in readmes
with open(f'{FOLDER_NAME}/readmes.txt', 'r') as f:
    readmes = eval(f.read())

In [3]:
main_prompt = "Rename the function specified in the code delimited by triple backticks to a more meaningful name that reflects its usage and/or aligns with the project's purpose. Do not add, remove, or change anything else. Structure your response under the following headings: 'New function name' (the new function name), 'Updated code' (the code with the function renamed), and 'Explanation' (a 1-2 sentence explanation of the new function name)."

ex1_input = """Project purpose:
This project is about analyzing top movie genre trends from 2000 to 2010. We will look at the top genres and top movies for each year.

Function to rename:
dat

Code:
```python
import pandas as pd

def dat(file_name):
    data = pd.read_csv(file_name)
    return data.head()

dat('data.csv')
```"""

ex1_output = """First, we look at how the identified function is being used. 'dat' is being used to read a csv file and return the first few rows of the dataframe. The function name 'dat' does not reflect its usage or align with the project's purpose. We can rename the function to 'get_movie_data_df' to reflect its usage and align with the project's purpose.

New function name:
get_movie_data_df

Now, we rename the function in the code. We need to update the function name in the function definition and in the function call. We output the full code below with the function renamed.

Updated code:
```python
import pandas as pd

def get_movie_data_df(file_name):
    data = pd.read_csv(file_name)
    return data.head()

get_movie_data_df('data.csv')
```

Now, let's briefly explain the new function name to the reader.

Explanation:
The new function name 'get_movie_data_df' reflects its usage to get a dataframe containing movie data."""

ex2_input = """Project purpose:
The focus of this project is determining the most common crime types in LA.

Function to rename:
group

Code:
```python
groups = df.groupby('gender')

def group(groups):
    for gender, grouped_data in groups:
        print(f"Gender: {gender}")
        print(grouped_data['crime_type'].value_counts())
        print("\n")
```"""

ex2_output = """First, we look at how the identified function is being used. 'group' is being used to print crime data grouped by gender. While the name 'group' reflects its usage, it does not reflect the project's purpose. A better function name which reflects this is 'print_group_crime_by_gender'.

New function name:
print_group_crime_by_gender

Now, we rename the function in the code. We need to update the function name in its definition only since it is not called. We output the full code below with the function renamed.

Updated code:
```python
groups = df.groupby('gender')

def print_group_crime_by_gender(groups):
    for gender, grouped_data in groups:
        print(f"Gender: {gender}")
        print(grouped_data['crime_type'].value_counts())
        print("\n")
```

Now, let's briefly explain the new function name to the reader.

Explanation:
The new function name 'print_group_crime_by_gender' aligns with its purpose of printing the crime data grouped by gender."""

ex3_input = """Project purpose:
This repository contains a collection of graph theory assignments for MAT381.

Function to rename:
visit

Code:
```python
def visit(graph, vertex):
    for neighbor in graph[vertex]:
        if neighbor not in visited:
            queue.append(neighbor)
            visited.add(neighbor)

while queue:
    vertex = queue.popleft()
    print(vertex, end=' ')
    visit(graph, vertex)
```"""

ex3_output = """First, we look at how the identified function is being used. 'visit' is being used to visit vertices in a graph. The name 'visit' reflects its usage, but it does not reflect the project's purpose. A better function name which reflects this is 'visit_vertex'.

New function name:
visit_vertex

Now, we rename the function in the code. We need to update the function name in its definition and in its call in the while-loop. We output the full code below with the function renamed.

Updated code:
```python
def visit_vertex(graph, vertex):
    for neighbor in graph[vertex]:
        if neighbor not in visited:
            queue.append(neighbor)
            visited.add(neighbor)

while queue:
    vertex = queue.popleft()
    print(vertex, end=' ')
    visit_vertex(graph, vertex)
```

Now, let's briefly explain the new function name to the reader.

Explanation:
By renaming visit to visit_vertex, it is now clearer that the function is being used to visit vertices."""

ex4_input = """Project purpose:
This project is for analyzing the relationship between the number of hours studied and the exam scores of students.

Function to rename:
get

Code:
```python
def get(x, y):
    return x.corr(y)

result, info = get(queue['Hours'], queue['Scores'])
print(result)
```"""

ex4_output = """First, we look at how the identified function is being used. 'get' is being used to calculate the correlation between the number of hours studied and the exam scores of students. The name 'get' does not reflect its usage or align with the project's purpose. We can rename the function to 'calc_hours_scores_correlation' to reflect its usage and align with the project's purpose.

New function name:
calc_hours_scores_correlation

Now, we rename the function in the code. We need to update the function name in the function definition and in the function call. We output the full code below with the function renamed.

Updated code:
```python
def calc_hours_scores_correlation(x, y):
    return x.corr(y)

result, info = calc_hours_scores_correlation(queue['Hours'], queue['Scores'])
print(result)
```

Now, let's briefly explain the new function name to the reader.

Explanation:
The function name 'calc_hours_scores_correlation' reflects the purpose of the function, which is to calculate the correlation between the number of hours studied and the exam scores of students."""

identify_vote_prompt = """Given an original task and multiple choices, choose the best answer for the original task. Analyze each choice in detail, then conclude in the last line 'The best choice is {s}', where s is the integer id of the choice. If all choices are equally good, return the smallest id. If no choice is good, return 0.
Original task: Suggest a more meaningful name for the function specified in the code delimited by triple backticks that reflects its usage and/or aligns with the project's purpose."""

identify_vote_ex1_input = """Project purpose:
This project is about analyzing top movie genre trends from 2000 to 2010. We will look at the top genres and top movies for each year.

Function to rename:
dat

Code:
```python
import pandas as pd

def dat(file_name):
    data = pd.read_csv(file_name)
    return data.head()

dat('data.csv')
```

Choice 1:
get_data
Choice 2:
get_movie
Choice 3:
get_movie_data_df"""

identify_vote_ex1_output = """Choice 1 is not the best choice. 'get_data' is a very generic name and does not reflect the project's purpose.
Choice 2 is also not the best choice. While 'get_movie' hints at the project's purpose, it does not reflect the function's usage.
Choice 3 is the best choice. 'get_movie_data_df' reflects the function's usage and aligns with the project's purpose.

The best choice is 3."""

code_vote_prompt = """Given an original task and multiple choices, choose the best answer for the original task. Analyze each choice in detail, then conclude in the last line 'The best choice is {s}', where s is the integer id of the choice. If all choices are equally good, return the smallest id. If no choice is good, return 0.
Original task: Rename the function specified in the code delimited by triple backticks to the new name specified. Update the function name everywhere it is referenced."""

code_vote_ex1_input = """Rename the function 'get' to 'calc_hours_scores_correlation' in the code delimited by triple backticks. Do not change anything else.
Original code:
```python
def get(x, y):
    return x.corr(y)

result, info = get(queue['Hours'], queue['Scores'])
print(result)
```

Choice 1:
```python
def calc_hours_scores_correlation(x, y):
    return x.corr(y)

result, info = calc_hours_scores_correlation(queue['Hours'], queue['Scores'])
print(result)
```
Choice 2:
```python
def calc_hours_scores_correlation(x, y):
    return x.corr(y)
```
Choice 3:
```python
def calc_hours_scores_correlation(x, y):
    return x.corr(y)

result, info = get(queue['Hours'], queue['Scores'])
print(result)
```
"""

code_vote_ex1_output = """Choice 1 is the best choice. It renames the function 'get' to 'calc_hours_scores_correlation' in the code delimited by triple backticks and everywhere it is referenced. It does not change anything else.
Choice 2 is incorrect. While it correctly renames the function 'get' to 'calc_hours_scores_correlation', it removes the other code outside the function definition. This is not specified in the prompt.
Choice 3 is incorrect. While it renames the function 'get' to 'calc_hours_scores_correlation' in its definition, it does not rename the function when it is later referenced.

The best choice is 1."""

def get_cot_prompt(purpose, name, cell_src):
    return [
        {"role": "user", "content": main_prompt},
        {"role": "user", "content": ex1_input},
        {"role": "assistant", "content": ex1_output},
        {"role": "user", "content": ex2_input},
        {"role": "assistant", "content": ex2_output},
        {"role": "user", "content": ex3_input},
        {"role": "assistant", "content": ex3_output},
        {"role": "user", "content": ex4_input},
        {"role": "assistant", "content": ex4_output},
        {"role" : "user", "content" : f"Project purpose:\n{purpose}\n\nFunction to rename:\n{name}\n\nCode:\n```python\n{cell_src}\n```"},
    ]

def get_identified_names(identify_trials, identify_completions):
    identified_names = []
    for i in range(identify_trials):
        if identify_completions.choices[i].finish_reason == 'stop':
            try:
                name = identify_completions.choices[i]['message']['content'].split('New function name:')[1].strip("\n").split("\n")[0]
            except:
                print("unexpected format for name", identify_completions.choices[i]['message']['content'])
                name = None
            else:
                if len(name.split('`')) == 3:
                    name = name.split('`')[1]
        else:
            name = None
        identified_names.append(name)
    return identified_names

def get_identify_vote_msgs(purpose, original_name, cell_src):
    def func(choices):
        final_msg = f"Project purpose:\n{purpose}\n\nFunction to rename:\n{original_name}\n\nCode:\n```python\n{cell_src}\n```\n\n"

        for i, choice in enumerate(choices):
            final_msg += f"Choice {i + 1}:\n{choice}\n"
        
        return [
        {"role": "user", "content": identify_vote_prompt},
        {"role": "user", "content": identify_vote_ex1_input},
        {"role": "assistant", "content": identify_vote_ex1_output},
        {"role": "user", "content": final_msg}
        ]
    
    return func

def get_code_vote_msgs(original_name, cell_src):
    def func(new_name, choices):
        final_msg = f"Rename the function '{original_name}' to '{new_name}' in the code delimited by triple backticks. Do not change anything else.\nOriginal code:\n```python\n{cell_src}\n```\n\n"
        
        for i, choice in enumerate(choices):
            final_msg += f"Choice {i + 1}:\n{choice}\n"
        
        return [
        {"role": "user", "content": code_vote_prompt},
        {"role": "user", "content": code_vote_ex1_input},
        {"role": "assistant", "content": code_vote_ex1_output},
        {"role" : "user", "content" : final_msg}
        ]

    return func

In [10]:
# GPT Tree of Thought
import sys
sys.path.append('../../../')
from tree_of_thought import solve_toc

identify_trials = 5
code_trials = 3
identify_vote_trials = 5
code_vote_trials = 5
identify_stop = "Updated code"

# identify and remove unused using GPT
gpt_results = []
for i in range(NUM_FILES):
    input_msgs_cot = get_cot_prompt(readmes[i], 'function_def', random_cells[i])
    get_identified_names_func = get_identified_names
    get_identify_votes_msgs_func = get_identify_vote_msgs(readmes[i], 'function_def', random_cells[i])
    get_code_votes_msgs_func = get_code_vote_msgs('function_def', random_cells[i])

    print(f'Processing file {i}')
    identified, updated_code = solve_toc(input_msgs_cot, identify_trials, code_trials, identify_vote_trials, code_vote_trials, identify_stop, get_identified_names_func, get_identify_votes_msgs_func, get_code_votes_msgs_func)
    print(f'File {i} - {identified}')
    gpt_results.append({'identified': identified, 'updated_code': updated_code})

# save the results to a file
with open(GPT_SAVED_FILE_NAME, 'w') as f:
    f.write(str(gpt_results))

Processing file 0
INFO: Depth 1: identified names are the same for all trials so we don't vote
INFO: Depth 2: updated code is the same for all trials so we don't vote
File 0 - mcculloch_pitts_neuron
Processing file 1
unexpected format for name First, we need to understand the purpose of the function based on its usage. However, the given code snippet does not provide any information about how the function is being used or what it is intended to do. Therefore, we cannot determine a more meaningful name for the function without additional context. 

To provide a more accurate suggestion for the function name, please provide more information about how the function is used or what it is intended to do in the project.
unexpected format for name First, we look at how the identified function is being used. 'function_def' is a placeholder function that does not have any functionality specified in the given code. The current function name 'function_def' does not reflect any specific purpose or 

In [11]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [12]:
# save the results to a variable
gpt_new_names = [var['identified'] for var in gpt_results]
gpt_new_code = [var['updated_code'] for var in gpt_results]

In [13]:
# Count the number of times the name is/isn't successfully changed
import sys
sys.path.append('../../determining_files_rename')
from ast_determine_usable_items import compare_code

pass_count = 0
fail_count = 0

for i in range(NUM_FILES):
    if compare_code(random_cells[i], gpt_new_code[i], 'function_def', gpt_new_names[i]):
        pass_count += 1
    else:
        fail_count += 1
        print(f'Failed on file {i}')

print(f'Pass count: {pass_count}, {pass_count / (pass_count + fail_count) * 100}%')
print(f'Fail count: {fail_count}, {fail_count / (pass_count + fail_count) * 100}%')

Failed on file 8
Pass count: 19, 95.0%
Fail count: 1, 5.0%
